# Extracting and Storing Highest-Grossing Films Data

### This script extracts information about the highest-grossing films from Wikipedia, processes the data, and saves it in both a SQLite database and a JSON file.

In [1]:
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import time
import re

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

response = requests.get(URL)
if response.status_code != 200:
    print("request error", response.status_code)
    exit()

## Web Scraping Approach

### 1. **Fetching the Wikipedia Page**
I use `requests.get(URL)` to send an HTTP request and check for a successful response (`status_code == 200`).

### 2. **Extracting the Main Table**
I use `BeautifulSoup`for locating the table and extracting all rows of the first table.

### 3. **Extracting Film Details**
For each row I extract the **title** and the link to the film's page and then visit the film's Wikipedia page and extract information below by searching for specific table headers (`th`) and extracting text from their corresponding `td` elements:
  - **Release Year** (`bday` class)
  - **Director(s)** (under the "Directed by" table row)
  - **Box Office Earnings** (under the "Box office" table row)
  - **Country** (under the "Country" table row)

### 4. **Handling Errors & Delays**
- **Error Handling:** I use `try-except` to safe script from stop.
- **Delays:** I didn't use delay before and it overwhelmed Wikipedia servers and I was blocked, so now I use`time.sleep(1)`.

In [3]:
soup = BeautifulSoup(response.text, "html.parser")

films_table = soup.find("table", class_="wikitable")
if not films_table:
    exit()

rows = films_table.find_all("tr")[1:] 

data = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 5:
        continue

    title_tag = row.find("th", scope="row").find("a") if row.find("th", scope="row") else None
    if not title_tag:
        continue

    title = title_tag.get_text(strip=True)
    film_url = "https://en.wikipedia.org" + title_tag["href"]
    
    print(f"Парсим {title} -> {film_url}")

    try:
        film_response = requests.get(film_url)
        film_soup = BeautifulSoup(film_response.text, "html.parser")
        

        year_tag = film_soup.find("span", class_="bday")
        release_year = year_tag.get_text(strip=True)[:4] if year_tag else "0000"

        director_tag = film_soup.find("th", string="Directed by")
        director = "Unknown"
        if director_tag:
            director_data = director_tag.find_next_sibling("td")
            if director_data:
                director = ", ".join([a.get_text(strip=True) for a in director_data.find_all("a")]) or director_data.get_text(strip=True)

        box_office_tag = film_soup.find("th", string="Box office")
        box_office = "Unknown"
        if box_office_tag:
            box_office_data = box_office_tag.find_next_sibling("td")
            box_office = box_office_data.get_text(strip=True) if box_office_data else "Unknown"

        country_tag = film_soup.find("th", string="Country")
        country = "Unknown"
        if country_tag:
            country_data = country_tag.find_next_sibling("td")
            if country_data:
                country = ", ".join([a.get_text(strip=True) for a in country_data.find_all("a")]) or country_data.get_text(strip=True)


        data.append((title, int(release_year), director, box_office, country))
    
    except Exception as e:
        print(f"Error {title}: {e}")

    time.sleep(1)

Парсим Avatar -> https://en.wikipedia.org/wiki/Avatar_(2009_film)
Парсим Avengers: Endgame -> https://en.wikipedia.org/wiki/Avengers:_Endgame
Парсим Avatar: The Way of Water -> https://en.wikipedia.org/wiki/Avatar:_The_Way_of_Water
Парсим Titanic -> https://en.wikipedia.org/wiki/Titanic_(1997_film)
Парсим Star Wars: The Force Awakens -> https://en.wikipedia.org/wiki/Star_Wars:_The_Force_Awakens
Парсим Avengers: Infinity War -> https://en.wikipedia.org/wiki/Avengers:_Infinity_War
Парсим Ne Zha 2 -> https://en.wikipedia.org/wiki/Ne_Zha_2
Парсим Spider-Man: No Way Home -> https://en.wikipedia.org/wiki/Spider-Man:_No_Way_Home
Парсим Inside Out 2 -> https://en.wikipedia.org/wiki/Inside_Out_2
Парсим Jurassic World -> https://en.wikipedia.org/wiki/Jurassic_World
Парсим The Lion King -> https://en.wikipedia.org/wiki/The_Lion_King_(2019_film)
Парсим The Avengers -> https://en.wikipedia.org/wiki/The_Avengers_(2012_film)
Парсим Furious 7 -> https://en.wikipedia.org/wiki/Furious_7
Парсим Top Gun: 

## Check for empty data
Few times I faced problem with empty json, so added this check.

In [4]:
if not data:
    exit()

## Data Cleaning 
**I didn't need to clean anything, except Box Office Data.**

Wikipedia often contains footnotes (`[1]`, `[2]`), empty boxes, and inconsistent formatting.
The function `clean_box_office()` removes these artifacts using:
  - `re.sub(r"\[.*?\]", "", value)` to remove footnotes.
  - `re.sub(r"(\d)(?=[mbMB])", r"\1 ", value)` for proper spacing.

In [5]:
def clean_box_office(value):
    if value == "Unknown":
        return value
    value = re.sub(r"\[.*?\]", "", value)  
    value = re.sub(r"(\d)(?=[mbMB])", r"\1 ", value)  
    return value.strip()

data = [(title, release_year, director, clean_box_office(box_office), country) for title, release_year, director, box_office, country in data]


## Storing Data in SQLite Database
### 1. **Creating the Database & Table**

I use SQLite (`sqlite3`) to create a `films.db` database with a table and then `executemany()` to insert all rows:
  - `title` (TEXT, NOT NULL)
  - `release_year` (INEGER)
  - `director` (TEXT)
  - `box_office` (TEXT)
  - `country` (TEXT)

In [6]:
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year TEXT,
    director TEXT,
    box_office TEXT,
    country TEXT
)
""")

cursor.executemany("INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?)", data)

conn.commit()
conn.close()


## Exporting Data to JSON

I used `json.dump()` with `ensure_ascii=False` to preserve non-ASCII characters in case there are non-English film titles.


In [7]:
with open("films.json", "w", encoding="utf-8") as f:
    json.dump([{"title": d[0], "release_year": d[1], "director": d[2], "box_office": d[3], "country": d[4]} for d in data], f, indent=4, ensure_ascii=False)
